In [1]:
#### import keras
from keras.applications import VGG16, VGG19
from keras.models import Model
from keras.layers.core import Activation, Dense
import tensorflow as tf
from keras.models import Sequential, ask_to_proceed_with_overwrite
from keras.layers import Dense
from keras.layers.core import Activation, Dense
from keras.layers import SimpleRNN, LSTM,  GRU,SeparableConvolution2D, Conv2D
from keras.callbacks import ModelCheckpoint,EarlyStopping
from keras.layers import Dense, GlobalAveragePooling2D

import glob
import scipy
import numpy as np
from PIL import Image
import random
import pandas as pd


def getModel( output_dim ):
    ''' 
        * output_dim: the number of classes (int)
        
        * return: compiled model (keras.engine.training.Model)
    '''
    #vgg_model = VGG16( weights='imagenet', include_top=False )
    vgg_model = VGG16(weights='imagenet', include_top=False)

    for layer in vgg_model.layers[:-3]:
        layer.trainable = False

    x = vgg_model.output
    x = GlobalAveragePooling2D()(x)
    # let's add a fully-connected layer
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(output_dim ,activation='softmax')(x)

# this is the model we will train
    model = Model(input=vgg_model.input, output=predictions)

    #Freeze all layers of VGG16 and Compile the model
    #Confirm the model is appropriate

    return model

def load_model(weights_path, output_dim):

    model = getModel( output_dim ) 
    for k,layer in list(model.layers_by_depth.items())[:]:
        layer[0].trainable = False
    
    model.load_weights(weights_path)
    model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', 'categorical_crossentropy'])   
    return model

Using TensorFlow backend.


In [2]:
model = load_model("weights-improvement-CPU-VGG16-LOGO7-02-0.8741.hdf5", 7)

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/keras/legacy/interfaces.py:86: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  '` call to the Keras 2 API: ' + signature)


In [3]:
## Load video dataset

In [3]:
path = "/Users/francescoferrari/Downloads/video_dataset_6/*/*/*.jpg"

files = sorted(glob.glob(path ))


In [11]:
len(files)/100

2869.85

In [12]:
initial_top_frames = {}

In [4]:
def storescores(new_lines):

    hs = open("top_frames_LOGO7.txt","a")
    for line in new_lines:
        f = line[0]
        v = line[1]
        v = ",".join([str(x) for x in v])
        hs.write(f+","+v+"\n")
    
    hs.close() 

In [21]:
files.index("/Users/francescoferrari/Downloads/video_dataset_6/Samsung/zdgn0qXrR8c/f_0149.jpg")

185

In [19]:
[0,1,2,3,4][1:]

[1, 2, 3, 4]

In [22]:
import time
classes_preds = []
avg_time = 87.0

limit = 185

#for i in range(0,len(files)-limit,limit):
for i in range(286800, 286985):
    start = time.time()
    imgs = []
    tmp_files = files[i:i+limit]
    for f in tmp_files:
        imgs.append(np.asarray(Image.open(f).resize((224, 224), Image.ANTIALIAS))/255.)
    print("loaded imgs")
    imgs = np.array(imgs)  - 0.468234346034
    tmp_preds = model.predict(imgs)
    storescores(list(zip(tmp_files, tmp_preds)))
    classes_preds += list(tmp_preds)
    avg_time = ((time.time()- start )+avg_time)/2.0
    remaing_epochs = (len(files) - (i+limit))/limit
    print("ETA: ", remaing_epochs*avg_time)
    print("finished preds " + str(i))

loaded imgs
ETA:  0.0
finished preds 286800
loaded imgs


KeyboardInterrupt: 

In [17]:
list(zip(tmp_files,tmp_preds))[0]

('/Users/francescoferrari/Downloads/video_dataset_6/Cocacola/9IG3JZhFlDo/f_0095.jpg',
 array([ 0.04795098,  0.04039044,  0.00346812,  0.12294032,  0.04654041,
         0.29538289,  0.44332686], dtype=float32))

In [23]:
len(classes_preds)

185

In [8]:
imgs = np.array(imgs)

In [ ]:
imgs -= imgs.mean()

In [ ]:
classes_preds = model.predict(imgs)

In [14]:
classes_preds

array([[  8.80705416e-01,   1.03876917e-02,   6.80321306e-02,
          4.08748053e-02],
       [  8.76025677e-01,   1.02994982e-02,   7.00935647e-02,
          4.35812883e-02],
       [  3.02785605e-01,   5.14376499e-02,   2.07867444e-01,
          4.37909305e-01],
       ..., 
       [  3.83556426e-01,   4.36403394e-01,   2.76346738e-03,
          1.77276701e-01],
       [  3.23374420e-01,   6.25799775e-01,   2.08877912e-03,
          4.87369746e-02],
       [  2.15747565e-01,   7.58411527e-01,   5.57620660e-04,
          2.52832230e-02]], dtype=float32)

In [15]:
len(classes_preds)

22588

In [7]:
r = zip(files[:len(classes_preds)],classes_preds)

In [17]:
f = open("frames_softmaxes_LOGO4.csv", "w")

In [20]:
for row in r:
    f.write(row[0] + "," + ",".join([str(x) for x in row[1]]) + "\n")

In [21]:
f.close()

In [24]:
r = []
f = open("top_frames_LOGO7.txt", "r")
for l in f:
    l = l[:-1].split(",")
    row = np.array([float(x) for x in l[1:]])
    r.append((l[0], row))
    

In [25]:
r_d = dict(r)

In [26]:
len(r_d)

286985

In [30]:
r_d

{'/Users/francescoferrari/Downloads/video_dataset_6/Nike/tCllnkLFmjU/f_0041.jpg': array([  2.83521000e-14,   8.24935000e-21,   4.93494000e-31,
          6.13249000e-18,   1.98070000e-15,   4.29569000e-18,
          1.00000000e+00]),
 '/Users/francescoferrari/Downloads/video_dataset_6/Pepsi/aZqjL5CNCd4/f_0303.jpg': array([  3.48723000e-01,   5.24496000e-03,   1.48263000e-04,
          4.92286000e-02,   5.57349000e-01,   5.11767000e-03,
          3.41886000e-02]),
 '/Users/francescoferrari/Downloads/video_dataset_6/Samsung/71kW-ZuGeeY/f_6035.jpg': array([  2.26505000e-04,   1.22683000e-02,   3.55405000e-06,
          1.00992000e-04,   3.64806000e-03,   9.83749000e-01,
          3.94330000e-06]),
 '/Users/francescoferrari/Downloads/video_dataset_6/LouisVuitton/2HBM5VB9uvY/f_0625.jpg': array([ 0.0747621,  0.0283928,  0.0543561,  0.197956 ,  0.028201 ,
         0.596296 ,  0.020036 ]),
 '/Users/francescoferrari/Downloads/video_dataset_6/Samsung/71kW-ZuGeeY/f_10104.jpg': array([ 0.0459196 , 

In [27]:
max_frames = 15

In [28]:
#brands_dict = {'Cocacola': 0, 'Nike': 2, 'Pepsi': 1}
brands_dict = {"Cocacola":0, "LOreal":1,"LouisVuitton":2,"Nike":3,"Pepsi":4,"Samsung":5}

In [37]:
brands_paths = sorted(glob.glob("/Users/francescoferrari/Downloads/video_dataset_6/*"))

top_frames = []

for brand in brands_paths:
    videos_paths = sorted(glob.glob(brand +"/*"))
    brand_label = brands_dict[brand.split("/")[-1]]
    for video in videos_paths:
        frames = sorted(glob.glob(video + "/*.jpg"))
        if len(frames) < max_frames:
            continue
        softmaxes = np.array([r_d[f] for f in frames])[:,brand_label]
        
        avgs = []
        for i in range(0, len(softmaxes)-max_frames):
            avgs.append(softmaxes[i:i+max_frames].mean())
        try:
            idx_max = np.argmax(avgs)
            top_frames.append((video,frames[idx_max] ))
        except ValueError:
            print(frames)
        

['/Users/francescoferrari/Downloads/video_dataset_6/Cocacola/mXWl5Qf5loU/f_0001.jpg', '/Users/francescoferrari/Downloads/video_dataset_6/Cocacola/mXWl5Qf5loU/f_0002.jpg', '/Users/francescoferrari/Downloads/video_dataset_6/Cocacola/mXWl5Qf5loU/f_0003.jpg', '/Users/francescoferrari/Downloads/video_dataset_6/Cocacola/mXWl5Qf5loU/f_0004.jpg', '/Users/francescoferrari/Downloads/video_dataset_6/Cocacola/mXWl5Qf5loU/f_0005.jpg', '/Users/francescoferrari/Downloads/video_dataset_6/Cocacola/mXWl5Qf5loU/f_0006.jpg', '/Users/francescoferrari/Downloads/video_dataset_6/Cocacola/mXWl5Qf5loU/f_0007.jpg', '/Users/francescoferrari/Downloads/video_dataset_6/Cocacola/mXWl5Qf5loU/f_0008.jpg', '/Users/francescoferrari/Downloads/video_dataset_6/Cocacola/mXWl5Qf5loU/f_0009.jpg', '/Users/francescoferrari/Downloads/video_dataset_6/Cocacola/mXWl5Qf5loU/f_0010.jpg', '/Users/francescoferrari/Downloads/video_dataset_6/Cocacola/mXWl5Qf5loU/f_0011.jpg', '/Users/francescoferrari/Downloads/video_dataset_6/Cocacola/mXWl

In [38]:
f = open("top_frames_LOGO7_frames10.csv", "w")

In [39]:
for l in top_frames:
    f.write(",".join(l)+"\n")

In [40]:
f.close()